In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0



geopy-1.21.0         | 58 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Lib

Finding geographical coordinates for Atlanta, GA USA

In [4]:
address = 'Atlanta, GA'

geolocator = Nominatim(user_agent="atl_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Atlanta are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Atlanta are 33.7490987, -84.3901849.


Creating the links to download the csv files I found on the Internet for this project, specifically zip code and neighborhood info

In [5]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import os

filepath ='https://docs.google.com/spreadsheets/d/e/2PACX-1vSZaqlui-2xMDWWeYkzqOaxFhdheFsGUpTZV8MUkjX7vWaaGda3t2VI8t7B9HohBkYivizJQVsFN-od/pub?gid=674492145&single=true&output=csv'
filepath2 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQeflARQnXsBdO-Orrrj61liwG9yP_UCU3jGPozeigSzUiOv5sqofoxphwIcgfPjpZBnInz7FjjKjdS/pub?gid=781352977&single=true&output=csv'

In [7]:
df = pd.read_csv(filepath)
df = df.filter(['Zip','Neighborhood'], axis=1)
df

,Zip,Neighborhood
0,30002,Avondale Estates
1,30021,Clarkston
2,30030,City of Decatur/Oakhurst/Winnona Park
3,30032,East Lake / South Decatur
4,30033,North Decatur
5,30080,Smyrna
6,30084,Tucker
7,30303,Downtown / Central Business District / Fairle...
8,30305,Buckhead / Tuxedo Park / Garden Hills / Hayne...
9,30306,Virginia Highlands / Morningside / Lenox Park...


Looking at zip codes in the entire state of Georgia to ensure I'm capturing the relevant zip codes in the metro Atlanta area

In [8]:
df2 = pd.read_csv(filepath2)
df2.head()

,Zip,Latitude,Longitude
0,31150,33.844371,-84.474050
1,31646,30.518903,-82.080630
2,31826,32.803518,-84.693840
3,30334,33.702657,-84.439127
4,31606,30.802411,-83.196240


In [9]:
atlanta = pd.merge(left=df, right=df2, left_on='Zip', right_on='Zip')
atlanta

,Zip,Neighborhood,Latitude,Longitude
0,30002,Avondale Estates,33.772122,-84.26491
1,30021,Clarkston,33.809604,-84.23870
2,30030,City of Decatur/Oakhurst/Winnona Park,33.771705,-84.29407
3,30032,East Lake / South Decatur,33.740055,-84.26791
4,30033,North Decatur,33.811354,-84.28335
5,30080,Smyrna,33.875780,-84.50382
6,30084,Tucker,33.853853,-84.22036
7,30303,Downtown / Central Business District / Fairle...,33.752856,-84.39013
8,30305,Buckhead / Tuxedo Park / Garden Hills / Hayne...,33.830054,-84.38472
9,30306,Virginia Highlands / Morningside / Lenox Park...,33.786755,-84.35149


Mapping Atlanta zip codes on city area map

In [10]:
map_atlanta = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(atlanta['Latitude'],atlanta['Longitude'], atlanta['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_atlanta)  
    
map_atlanta

In [11]:
CLIENT_ID = 'RM0IXCNGIMPPZ14RBHT0QMC4V20CNPUMQRMDWNC0X4PDH023' # your Foursquare ID
CLIENT_SECRET = 'FDXRQG4E3YVFLUVWJLJ12RVKBWD50LGHSYODHXXGUMOPZ0AG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RM0IXCNGIMPPZ14RBHT0QMC4V20CNPUMQRMDWNC0X4PDH023
CLIENT_SECRET:FDXRQG4E3YVFLUVWJLJ12RVKBWD50LGHSYODHXXGUMOPZ0AG


Imported my 4sq info and ensured that I could read from the API by checking the first neighborhood on the list

In [12]:
neighborhood_latitude = atlanta.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = atlanta.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = atlanta.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))



Latitude and longitude values of  Avondale Estates are 33.772121999999996, -84.26491.


In [16]:
LIMIT = 400 # limit of number of venues returned by Foursquare API



radius = 5000 # define radius



# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display UR


'https://api.foursquare.com/v2/venues/explore?&client_id=RM0IXCNGIMPPZ14RBHT0QMC4V20CNPUMQRMDWNC0X4PDH023&client_secret=FDXRQG4E3YVFLUVWJLJ12RVKBWD50LGHSYODHXXGUMOPZ0AG&v=20180605&ll=33.772121999999996,-84.26491&radius=5000&limit=400'

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9e30c77828ae001b50ebfe'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Avondale Estates',
  'headerFullLocation': 'Avondale Estates',
  'headerLocationGranularity': 'city',
  'totalResults': 216,
  'suggestedBounds': {'ne': {'lat': 33.81712204500004,
    'lng': -84.21087601234815},
   'sw': {'lat': 33.72712195499995, 'lng': -84.31894398765185}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '56f6a924498e9eff48634ecf',
       'name': 'Rising Son',
       'location': {'address': '124 N Avondale Rd',
        'lat': 33.77641685720219,
        'lng': -84.2675154031931,
        'labeledLatLngs': [{'label':

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Rising Son,Restaurant,33.776417,-84.267515
1,My Parents' Basement,Beer Bar,33.775612,-84.272334
2,Arepa Mia,Venezuelan Restaurant,33.776942,-84.267327
3,Lifeline Animal Project,Dog Run,33.776895,-84.269454
4,Good Karma Coffee House,Gluten-free Restaurant,33.776037,-84.265889


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:

atl_venues = getNearbyVenues(names=atlanta['Neighborhood'],
                                   latitudes=atlanta['Latitude'],
                                   longitudes=atlanta['Longitude']
                                  )



 Avondale Estates
 Clarkston
 City of Decatur/Oakhurst/Winnona Park
 East Lake / South Decatur
 North Decatur
 Smyrna
 Tucker 
 Downtown / Central Business District / Fairlee Poplar
 Buckhead / Tuxedo Park / Garden Hills / Haynes Manor / Peachtree Battle / Peachtree Hills
 Virginia Highlands / Morningside / Lenox Park / Poncey Highland / Druid Hills
 Candler Park / Druid Hills / Edgewood / Emory / Inman Park / Lake Claire / Little Five Points
 Midtown / Old Fourth Ward
 Midtown / Ansley Park / Brookwood Hills / Loring Heights
 Adair Park / Capitol View / Oakland City / West End
 Cascade
 Downtown Atlanta / Grant Park
 Downtown Atlanta / Castleberry Hills
 Vines City / Mozeley Park
 Grant Park peoplestown / Lakewood
 Cabbagetown / East Atlanta Village /  Ormewood Park /South Dekalb
 East Lake / Kirkwood / Edgewood
 Home Park /Northwest Atlanta / Collier Hills / Underwood hills / Midtown West
 Brookhaven / North Atlanta / Dunwoody
 Morningside/Lenox Park / Piedmont Heights / Lenox / Lavi

The code above was created to ensure I could get venue coordinates and types for future analysis.  Specifically, I ensured I could download the JSON file from 4sq, look at venues a certain distance from the zip code's coordinates, and categorize each entry.  Below you'll see this output:

In [16]:
atl_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Avondale Estates,33.772122,-84.26491,Rising Son,33.776417,-84.267515,Restaurant
1,Avondale Estates,33.772122,-84.26491,Good Karma Coffee House,33.776037,-84.265889,Gluten-free Restaurant
2,Avondale Estates,33.772122,-84.26491,Avondale Lake & Bess Walking Park,33.768739,-84.266118,Park
3,Avondale Estates,33.772122,-84.26491,Arepa Mia,33.776942,-84.267327,Venezuelan Restaurant
4,Avondale Estates,33.772122,-84.26491,Avondale Estates,33.775358,-84.267100,City


I'm now proceeding to normalize the data to determine the top venue categories in each neighborhood

In [25]:
# one hot encoding
atl_onehot = pd.get_dummies(atl_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
atl_onehot['Neighborhood'] = atl_venues['Neighborhood'] 

cols = list(atl_onehot.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('Neighborhood')) #Remove Neighborhood from list
atl_onehot = atl_onehot[['Neighborhood'] + cols] #Create new dataframe with Neighborhood as the first column
atl_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cemetery,Chinese Restaurant,Chiropractor,Chocolate Shop,Circus,City,Clothing Store,Cocktail Bar,Coffee Shop,College & University,College Basketball Court,College Bookstore,College Rec Center,College Theater,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Credit Union,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Insurance Office,Internet Cafe,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Paintball Field,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Piercing Parlor,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Pub,Public Art,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Taxi,Taxi Stand,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attracti

In [26]:
atl_grouped = atl_onehot.groupby('Neighborhood').mean().reset_index()
atl_grouped

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cemetery,Chinese Restaurant,Chiropractor,Chocolate Shop,Circus,City,Clothing Store,Cocktail Bar,Coffee Shop,College & University,College Basketball Court,College Bookstore,College Rec Center,College Theater,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Credit Union,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Insurance Office,Internet Cafe,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Paintball Field,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Piercing Parlor,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Pub,Public Art,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Taxi,Taxi Stand,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attracti

I'm now finding the top 20 venue types in each neighborhood to find any interesting insights to guide my business decision.  Specifically, I'm looking at places where one of the top destinations is a type of Asian restaurant and where it isn't to determine where my opportunities could lie and to keep in mind for further in the exercise.  I selected Asian restaurant after seeing the frequency and variety of them throughout the Atlanta database (sushi, vietnamese, Bubble Tea, Indian, etc.)

In [27]:
num_top_venues = 20

for hood in atl_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = atl_grouped[atl_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Adair Park / Capitol View / Oakland City / West End----
                              venue  freq
0                       Gas Station  0.06
1                              Park  0.05
2                    Cosmetics Shop  0.05
3     Vegetarian / Vegan Restaurant  0.05
4                       Pizza Place  0.05
5                    Discount Store  0.05
6               Fried Chicken Joint  0.05
7                              Bank  0.04
8                             Trail  0.04
9                    Sandwich Place  0.04
10                       Donut Shop  0.02
11             Caribbean Restaurant  0.02
12  Southern / Soul Food Restaurant  0.02
13               Seafood Restaurant  0.02
14                    Grocery Store  0.02
15               Chinese Restaurant  0.02
16             Fast Food Restaurant  0.02
17                Convenience Store  0.02
18           Thrift / Vintage Store  0.02
19                       Food Truck  0.01


---- Avondale Estates----
                     venue  f

Beginning the clustering exercise to segment the neighborhoods and find similarities.

In [36]:
# set number of clusters
kclusters = 5

atl_grouped_clustering = atl_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(atl_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 2, 3, 2, 0, 2, 2, 2, 2, 2], dtype=int32)

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = atl_grouped['Neighborhood']

for ind in np.arange(atl_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(atl_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted



,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adair Park / Capitol View / Oakland City / We...,Gas Station,Vegetarian / Vegan Restaurant,Park,Discount Store,Fried Chicken Joint,Pizza Place,Cosmetics Shop,Bank,Sandwich Place,Trail
1,Avondale Estates,Thrift / Vintage Store,Pizza Place,Bakery,Brewery,Gas Station,Sandwich Place,Athletics & Sports,Theater,Park,Pub
2,Briarcliff Woods / Oak Grove / Northlake,Pool,Coffee Shop,Sandwich Place,Park,Intersection,Nature Preserve,Grocery Store,Shipping Store,Gas Station,Gym
3,Brookhaven / North Atlanta / Dunwoody,American Restaurant,Pizza Place,Sandwich Place,Mexican Restaurant,Health & Beauty Service,Chinese Restaurant,Sports Bar,Furniture / Home Store,BBQ Joint,Frozen Yogurt Shop
4,Buckhead / North Atlanta,American Restaurant,Mexican Restaurant,Grocery Store,Sporting Goods Shop,Coffee Shop,Gas Station,Smoothie Shop,Jewelry Store,Salon / Barbershop,Bagel Shop
5,Buckhead / North Buckhead / Chastain Park / N...,Park,Mexican Restaurant,Wings Joint,Bakery,Gym,BBQ Joint,Grocery Store,Italian Restaurant,Spa,Paper / Office Supplies Store
6,Buckhead / Tuxedo Park / Garden Hills / Hayne...,Steakhouse,American Restaurant,Pizza Place,Italian Restaurant,New American Restaurant,Grocery Store,Spa,Salon / Barbershop,Gastropub,Café
7,Cabbagetown / East Atlanta Village / Ormewoo...,Bar,Discount Store,Gas Station,Pizza Place,Park,Coffee Shop,Video Store,Bank,Sushi Restaurant,Fast Food Restaurant
8,Candler Park / Druid Hills / Edgewood / Emory...,Park,Pizza Place,Bar,Gift Shop,Bookstore,BBQ Joint,Coffee Shop,Garden,Italian Restaurant,Southern / Soul Food Restaurant
9,Cascade,Dry Cleaner,Park,Gas Station,Nature Preserve,Coffee Shop,Seafood Restaurant,Liquor Store,Chinese Restaurant,BBQ Joint,Golf Course


In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

atl_merged = atlanta

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
atl_merged = atl_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

atl_merged

,Zip,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,30002,Avondale Estates,33.772122,-84.26491,2,Thrift / Vintage Store,Pizza Place,Bakery,Brewery,Gas Station,Sandwich Place,Athletics & Sports,Theater,Park,Pub
1,30021,Clarkston,33.809604,-84.23870,3,Intersection,Discount Store,Chinese Restaurant,Sandwich Place,Grocery Store,Middle Eastern Restaurant,Fried Chicken Joint,Football Stadium,Food Truck,Breakfast Spot
2,30030,City of Decatur/Oakhurst/Winnona Park,33.771705,-84.29407,2,Pub,Gastropub,Pizza Place,Brewery,Ice Cream Shop,Breakfast Spot,Italian Restaurant,Park,Bakery,American Restaurant
3,30032,East Lake / South Decatur,33.740055,-84.26791,4,Fast Food Restaurant,Discount Store,Wings Joint,Pizza Place,Grocery Store,Gas Station,Seafood Restaurant,Chinese Restaurant,Fried Chicken Joint,Video Store
4,30033,North Decatur,33.811354,-84.28335,2,Indian Restaurant,Fast Food Restaurant,Cosmetics Shop,Grocery Store,Discount Store,American Restaurant,Mediterranean Restaurant,Restaurant,Fried Chicken Joint,Sandwich Place
5,30080,Smyrna,33.875780,-84.50382,1,Bar,Pizza Place,Park,Café,Library,Bookstore,Fried Chicken Joint,Liquor Store,Sushi Restaurant,Food Truck
6,30084,Tucker,33.853853,-84.22036,2,Pizza Place,Fast Food Restaurant,Wings Joint,Mexican Restaurant,Discount Store,Ice Cream Shop,Bank,Pharmacy,Pet Store,Mobile Phone Shop
7,30303,Downtown / Central Business District / Fairle...,33.752856,-84.39013,2,Hotel,Art Gallery,Park,Burger Joint,BBQ Joint,Hotel Bar,Fast Food Restaurant,Museum,Aquarium,History Museum
8,30305,Buckhead / Tuxedo Park / Garden Hills / Hayne...,33.830054,-84.38472,2,Steakhouse,American Restaurant,Pizza Place,Italian Restaurant,New American Restaurant,Grocery Store,Spa,Salon / Barbershop,Gastropub,Café
9,30306,Virginia Highlands / Morningside / Lenox Park...,33.786755,-84.35149,1,Bar,American Restaurant,Trail,Pizza Place,Italian Restaurant,Gym,Grocery Store,Burger Joint,Dessert Shop,Thai Restaurant


In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(atl_merged['Latitude'], atl_merged['Longitude'], atl_merged['Neighborhood'], atl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Knowing how the neighborhoods are clustered, I now want to analyze each cluster and compare this info to the top 20 venues info from the code above to help me narrow my decision.

In [37]:
atl_merged.loc[atl_merged['Cluster Labels'] == 0, atl_merged.columns[[1] + list(range(5, atl_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Buckhead / North Atlanta,American Restaurant,Mexican Restaurant,Grocery Store,Sporting Goods Shop,Coffee Shop,Gas Station,Smoothie Shop,Jewelry Store,Salon / Barbershop,Bagel Shop


In [38]:
atl_merged.loc[atl_merged['Cluster Labels'] == 1, atl_merged.columns[[1] + list(range(5, atl_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Smyrna,Bar,Pizza Place,Park,Café,Library,Bookstore,Fried Chicken Joint,Liquor Store,Sushi Restaurant,Food Truck
9,Virginia Highlands / Morningside / Lenox Park...,Bar,American Restaurant,Trail,Pizza Place,Italian Restaurant,Gym,Grocery Store,Burger Joint,Dessert Shop,Thai Restaurant


In [39]:
atl_merged.loc[atl_merged['Cluster Labels'] == 2, atl_merged.columns[[1] + list(range(5, atl_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Avondale Estates,Thrift / Vintage Store,Pizza Place,Bakery,Brewery,Gas Station,Sandwich Place,Athletics & Sports,Theater,Park,Pub
2,City of Decatur/Oakhurst/Winnona Park,Pub,Gastropub,Pizza Place,Brewery,Ice Cream Shop,Breakfast Spot,Italian Restaurant,Park,Bakery,American Restaurant
4,North Decatur,Indian Restaurant,Fast Food Restaurant,Cosmetics Shop,Grocery Store,Discount Store,American Restaurant,Mediterranean Restaurant,Restaurant,Fried Chicken Joint,Sandwich Place
6,Tucker,Pizza Place,Fast Food Restaurant,Wings Joint,Mexican Restaurant,Discount Store,Ice Cream Shop,Bank,Pharmacy,Pet Store,Mobile Phone Shop
7,Downtown / Central Business District / Fairle...,Hotel,Art Gallery,Park,Burger Joint,BBQ Joint,Hotel Bar,Fast Food Restaurant,Museum,Aquarium,History Museum
8,Buckhead / Tuxedo Park / Garden Hills / Hayne...,Steakhouse,American Restaurant,Pizza Place,Italian Restaurant,New American Restaurant,Grocery Store,Spa,Salon / Barbershop,Gastropub,Café
10,Candler Park / Druid Hills / Edgewood / Emory...,Park,Pizza Place,Bar,Gift Shop,Bookstore,BBQ Joint,Coffee Shop,Garden,Italian Restaurant,Southern / Soul Food Restaurant
11,Midtown / Old Fourth Ward,Southern / Soul Food Restaurant,American Restaurant,Hotel,Trail,Coffee Shop,Grocery Store,Seafood Restaurant,Pizza Place,Park,Mediterranean Restaurant
12,Midtown / Ansley Park / Brookwood Hills / Lor...,Coffee Shop,Pizza Place,American Restaurant,Café,Seafood Restaurant,Park,Hotel,Mexican Restaurant,Japanese Restaurant,Botanical Garden
14,Cascade,Dry Cleaner,Park,Gas Station,Nature Preserve,Coffee Shop,Seafood Restaurant,Liquor Store,Chinese Restaurant,BBQ Joint,Golf Course


In [40]:
atl_merged.loc[atl_merged['Cluster Labels'] == 3, atl_merged.columns[[1] + list(range(5, atl_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Clarkston,Intersection,Discount Store,Chinese Restaurant,Sandwich Place,Grocery Store,Middle Eastern Restaurant,Fried Chicken Joint,Football Stadium,Food Truck,Breakfast Spot
33,Briarcliff Woods / Oak Grove / Northlake,Pool,Coffee Shop,Sandwich Place,Park,Intersection,Nature Preserve,Grocery Store,Shipping Store,Gas Station,Gym


In [41]:
atl_merged.loc[atl_merged['Cluster Labels'] == 4, atl_merged.columns[[1] + list(range(5, atl_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Lake / South Decatur,Fast Food Restaurant,Discount Store,Wings Joint,Pizza Place,Grocery Store,Gas Station,Seafood Restaurant,Chinese Restaurant,Fried Chicken Joint,Video Store
13,Adair Park / Capitol View / Oakland City / We...,Gas Station,Vegetarian / Vegan Restaurant,Park,Discount Store,Fried Chicken Joint,Pizza Place,Cosmetics Shop,Bank,Sandwich Place,Trail
17,Vines City / Mozeley Park,Gas Station,Park,Trail,Southern / Soul Food Restaurant,Pizza Place,Fried Chicken Joint,Wings Joint,Nightclub,Caribbean Restaurant,Discount Store


After analyzing this data, I focused on cluster 0.  Anecdotally, Chamblee is a racially-diverse neighborhood, while North Decatur is in the vicinity of Emory University, an institution that attracts students from around the world - particularly Asians.  I find it surprising that, with this in mind, there isn't an Asian Restaurant in the top 10 or 20 venue categories in North Decatur, while Chamblee, a zip code in which Asians rank as the 4th largest ethnic group, has 4 types of Asian restaurants in the top 10 of most prevalent venues.  To verify this hypothesis, I decided to further cluster the neighborhoods in cluster 0 to determine if, within this group, I can further refine my findings and offer a more confident recommendation.

In [42]:
atl_clust_zero = atl_merged.loc[atl_merged['Cluster Labels'] == 2, atl_merged.columns[[1] + list(range(5, atl_merged.shape[1]))]]
atl_clust_zero

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Avondale Estates,Thrift / Vintage Store,Pizza Place,Bakery,Brewery,Gas Station,Sandwich Place,Athletics & Sports,Theater,Park,Pub
2,City of Decatur/Oakhurst/Winnona Park,Pub,Gastropub,Pizza Place,Brewery,Ice Cream Shop,Breakfast Spot,Italian Restaurant,Park,Bakery,American Restaurant
4,North Decatur,Indian Restaurant,Fast Food Restaurant,Cosmetics Shop,Grocery Store,Discount Store,American Restaurant,Mediterranean Restaurant,Restaurant,Fried Chicken Joint,Sandwich Place
6,Tucker,Pizza Place,Fast Food Restaurant,Wings Joint,Mexican Restaurant,Discount Store,Ice Cream Shop,Bank,Pharmacy,Pet Store,Mobile Phone Shop
7,Downtown / Central Business District / Fairle...,Hotel,Art Gallery,Park,Burger Joint,BBQ Joint,Hotel Bar,Fast Food Restaurant,Museum,Aquarium,History Museum
8,Buckhead / Tuxedo Park / Garden Hills / Hayne...,Steakhouse,American Restaurant,Pizza Place,Italian Restaurant,New American Restaurant,Grocery Store,Spa,Salon / Barbershop,Gastropub,Café
10,Candler Park / Druid Hills / Edgewood / Emory...,Park,Pizza Place,Bar,Gift Shop,Bookstore,BBQ Joint,Coffee Shop,Garden,Italian Restaurant,Southern / Soul Food Restaurant
11,Midtown / Old Fourth Ward,Southern / Soul Food Restaurant,American Restaurant,Hotel,Trail,Coffee Shop,Grocery Store,Seafood Restaurant,Pizza Place,Park,Mediterranean Restaurant
12,Midtown / Ansley Park / Brookwood Hills / Lor...,Coffee Shop,Pizza Place,American Restaurant,Café,Seafood Restaurant,Park,Hotel,Mexican Restaurant,Japanese Restaurant,Botanical Garden
14,Cascade,Dry Cleaner,Park,Gas Station,Nature Preserve,Coffee Shop,Seafood Restaurant,Liquor Store,Chinese Restaurant,BBQ Joint,Golf Course


In [45]:
atl_grouped_zero = pd.merge(left = atl_clust_zero, right= atl_grouped, left_on='Neighborhood', right_on='Neighborhood')
atl_grouped_zero = atl_grouped_zero.drop(['1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', '9th Most Common Venue', '10th Most Common Venue'], axis =1)
atl_grouped_zero

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cemetery,Chinese Restaurant,Chiropractor,Chocolate Shop,Circus,City,Clothing Store,Cocktail Bar,Coffee Shop,College & University,College Basketball Court,College Bookstore,College Rec Center,College Theater,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Credit Union,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Insurance Office,Internet Cafe,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Paintball Field,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Piercing Parlor,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Pub,Public Art,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Taxi,Taxi Stand,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attracti

Now that I've created a normalized data set with the neighborhoods in cluster 0, I'm going to do a k means clustering exercise with only 3 clusters to ensure I get meaningful groups.

In [60]:
kclusters1 = 3

atl_grouped_clustering_zero = atl_grouped_zero.drop('Neighborhood', 1)

# run k-means clustering
kmeans1 = KMeans(n_clusters=kclusters1, random_state=0).fit(atl_grouped_clustering_zero)

# check cluster labels generated for each row in the dataframe
kmeans1.labels_[0:10] 

array([0, 0, 2, 2, 0, 0, 0, 0, 0, 1], dtype=int32)

In [61]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cluster_zero_venues_sorted = pd.DataFrame(columns=columns)
cluster_zero_venues_sorted['Neighborhood'] = atl_grouped_zero['Neighborhood']

for ind in np.arange(atl_grouped_zero.shape[0]):
    cluster_zero_venues_sorted.iloc[ind, 1:] = return_most_common_venues(atl_grouped_zero.iloc[ind, :], num_top_venues)

cluster_zero_venues_sorted



,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Avondale Estates,Thrift / Vintage Store,Pizza Place,Bakery,Brewery,Gas Station,Sandwich Place,Athletics & Sports,Theater,Park,Pub
1,City of Decatur/Oakhurst/Winnona Park,Pub,Gastropub,Pizza Place,Brewery,Ice Cream Shop,Breakfast Spot,Italian Restaurant,Park,Bakery,American Restaurant
2,North Decatur,Indian Restaurant,Fast Food Restaurant,Cosmetics Shop,Grocery Store,Discount Store,American Restaurant,Mediterranean Restaurant,Restaurant,Fried Chicken Joint,Sandwich Place
3,Tucker,Pizza Place,Fast Food Restaurant,Wings Joint,Mexican Restaurant,Discount Store,Ice Cream Shop,Bank,Pharmacy,Pet Store,Mobile Phone Shop
4,Downtown / Central Business District / Fairle...,Hotel,Art Gallery,Park,Burger Joint,BBQ Joint,Hotel Bar,Fast Food Restaurant,Museum,Aquarium,History Museum
5,Buckhead / Tuxedo Park / Garden Hills / Hayne...,Steakhouse,American Restaurant,Pizza Place,Italian Restaurant,New American Restaurant,Grocery Store,Spa,Salon / Barbershop,Gastropub,Café
6,Candler Park / Druid Hills / Edgewood / Emory...,Park,Pizza Place,Bar,Gift Shop,Bookstore,BBQ Joint,Coffee Shop,Garden,Italian Restaurant,Southern / Soul Food Restaurant
7,Midtown / Old Fourth Ward,Southern / Soul Food Restaurant,American Restaurant,Hotel,Trail,Coffee Shop,Grocery Store,Seafood Restaurant,Pizza Place,Park,Mediterranean Restaurant
8,Midtown / Ansley Park / Brookwood Hills / Lor...,Coffee Shop,Pizza Place,American Restaurant,Café,Seafood Restaurant,Park,Hotel,Mexican Restaurant,Japanese Restaurant,Botanical Garden
9,Cascade,Dry Cleaner,Park,Gas Station,Nature Preserve,Coffee Shop,Seafood Restaurant,Liquor Store,Chinese Restaurant,BBQ Joint,Golf Course


In [62]:
cluster_zero_venues_sorted.insert(0, 'Cluster Labels', kmeans1.labels_)

Now that I can confirm that the new analysis yields the same output as the initial, I added zip code and lat/long information to prepare the data for the mapping exercise.

In [63]:


atl_merged_zero = atlanta


atl_merged_zero = pd.merge(left = atl_merged_zero, right = cluster_zero_venues_sorted, left_on='Neighborhood', right_on='Neighborhood')
atl_merged_zero

,Zip,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,30002,Avondale Estates,33.772122,-84.26491,0,Thrift / Vintage Store,Pizza Place,Bakery,Brewery,Gas Station,Sandwich Place,Athletics & Sports,Theater,Park,Pub
1,30030,City of Decatur/Oakhurst/Winnona Park,33.771705,-84.29407,0,Pub,Gastropub,Pizza Place,Brewery,Ice Cream Shop,Breakfast Spot,Italian Restaurant,Park,Bakery,American Restaurant
2,30033,North Decatur,33.811354,-84.28335,2,Indian Restaurant,Fast Food Restaurant,Cosmetics Shop,Grocery Store,Discount Store,American Restaurant,Mediterranean Restaurant,Restaurant,Fried Chicken Joint,Sandwich Place
3,30084,Tucker,33.853853,-84.22036,2,Pizza Place,Fast Food Restaurant,Wings Joint,Mexican Restaurant,Discount Store,Ice Cream Shop,Bank,Pharmacy,Pet Store,Mobile Phone Shop
4,30303,Downtown / Central Business District / Fairle...,33.752856,-84.39013,0,Hotel,Art Gallery,Park,Burger Joint,BBQ Joint,Hotel Bar,Fast Food Restaurant,Museum,Aquarium,History Museum
5,30305,Buckhead / Tuxedo Park / Garden Hills / Hayne...,33.830054,-84.38472,0,Steakhouse,American Restaurant,Pizza Place,Italian Restaurant,New American Restaurant,Grocery Store,Spa,Salon / Barbershop,Gastropub,Café
6,30307,Candler Park / Druid Hills / Edgewood / Emory...,33.768205,-84.33786,0,Park,Pizza Place,Bar,Gift Shop,Bookstore,BBQ Joint,Coffee Shop,Garden,Italian Restaurant,Southern / Soul Food Restaurant
7,30308,Midtown / Old Fourth Ward,33.771755,-84.38065,0,Southern / Soul Food Restaurant,American Restaurant,Hotel,Trail,Coffee Shop,Grocery Store,Seafood Restaurant,Pizza Place,Park,Mediterranean Restaurant
8,30309,Midtown / Ansley Park / Brookwood Hills / Lor...,33.794055,-84.38525,0,Coffee Shop,Pizza Place,American Restaurant,Café,Seafood Restaurant,Park,Hotel,Mexican Restaurant,Japanese Restaurant,Botanical Garden
9,30311,Cascade,33.724907,-84.47131,1,Dry Cleaner,Park,Gas Station,Nature Preserve,Coffee Shop,Seafood Restaurant,Liquor Store,Chinese Restaurant,BBQ Joint,Golf Course


In [64]:
# create map
map_cluster_zero = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters1)
ys = [i + x + (i*x)**2 for i in range(kclusters1)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(atl_merged_zero['Latitude'], atl_merged_zero['Longitude'], atl_merged_zero['Neighborhood'], atl_merged_zero['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_cluster_zero)
       
map_cluster_zero

The more detailed analysis shows that, indeed, Chamblee and North Decatur are in the same microcluster and, thus, that we can support our hypothesis with data.   